In [ ]:
from googleapiclient import discovery
from pathlib import Path
import json
import time
from tqdm import tqdm

from constants import TEXTS_DIR, PERSPECTIVE_API_RESPONSE_DIR, PERSPECTIVE_API_KEY

In [ ]:
# Generates API client object dynamically based on service name and version.
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=PERSPECTIVE_API_KEY)

def request_toxicity(text, response_path):
    analyze_request = {
      'comment': { 'text': text },
      'requestedAttributes': {'TOXICITY': {}}
    }

    response = service.comments().analyze(body=analyze_request).execute()

    with response_path.open('w') as f:
        json.dump(response, f)

In [ ]:
PERSPECTIVE_API_SLEEP_SECONDS = 1
PERSPECTIVE_API_LEN_LIMIT = 20480

text_paths = list(TEXTS_DIR.iterdir())
for text_file in tqdm_notebook(text_paths):
    response_filename = text_file.name + '.json'
    response_file = PERSPECTIVE_API_RESPONSE_DIR / response_filename
    if response_file.exists():
        continue

    text = text_file.read_text()
    if len(text) > PERSPECTIVE_API_LEN_LIMIT:
        continue

    try:
        request_toxicity(text, response_file)
    except:
        pass

    # Sleep for 1 second due to rate limiting by API
    time.sleep(PERSPECTIVE_API_SLEEP_SECONDS)